# Balancing Authority Example

The following example shows how to use the assetra package to analyze an existing energy system. California ISO is taken as a use-case with system-specific data sources.

### 1. Setup *assetra*

To begin, we initialize an `EnergySystemBuilder` object. We will add demand and generating units to this system builder in the following sections.

In [18]:
from assetra.system import EnergySystemBuilder

builder = EnergySystemBuilder()
unit_count = 0

### 2. Load Demand Data

The `assetra` interface is source agnostic, but in this example we use publicly-available historical hourly demand profiles based on the EIA-930 dataset. 

**Source:** https://github.com/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Data

In [2]:
from pathlib import Path
from datetime import datetime

import xarray as xr
import pandas as pd

def load_eia_930_cleaned_hourly_demand(
        eia_930_cleaned_demand_file: Path,
        start_hour: datetime,
        end_hour: datetime) -> xr.DataArray:
    """Return hourly demand data. This function expects a formatted csv which can
    be downloaded from:

    https://github.com/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Data
    """
    # read demand file
    eia_930_df = pd.read_csv(
        eia_930_cleaned_demand_file,
        usecols=["date_time", "cleaned demand (MW)"],
        index_col="date_time",
        parse_dates=True,
    )

    # keep demand
    hourly_demand_pd = eia_930_df["cleaned demand (MW)"].loc[start_hour:end_hour]

    # convert to xr.DataArray
    hourly_demand = xr.DataArray(
        data=hourly_demand_pd.values,
        coords=dict(
            time=hourly_demand_pd.index.values
        )
    )
    return hourly_demand

eia_930_cleaned_demand_file = Path('sample_data', 'CISO.csv')
hourly_demand = load_eia_930_cleaned_hourly_demand(
	eia_930_cleaned_demand_file,
	start_hour='2019-01-01 00:00:00',
	end_hour='2019-12-31 23:00:00'
)

The previous code block returns an hourly demand profile in an `xarray.DataArray` object. We can add this profile to our energy system as an `assetra.units.DemandUnit` object. 

**Notes:**

 - All objects inhering from `assetra.units.EnergyUnit` (including demand units) must have unique identifiying number. For the purposes of this example, we use an incrementing counter.

In [3]:
from assetra.units import DemandUnit

# start unit count
builder.add_unit(
    DemandUnit(
        id=unit_count,
        hourly_demand=hourly_demand
    )
)
unit_count += 1


### 2. Load Generator Data

We get unit-level generator data from EIA form-860. From this dataset, we identify nameplate capacity, location, and technology.

**Source:** https://www.eia.gov/electricity/data/eia860/

In [4]:
def load_eia_860_plants(eia_860_plant_file: Path, bal_auth: str) -> tuple[pd.Series, pd.Series, pd.Series]:
    """Return series of eia 860 plant codes, latitudes, and longitudes"""
    # read file
    eia_860_plant_df = pd.read_excel(
        eia_860_plant_file,
        skiprows=1,
        usecols=[
            "Plant Code",
            "Latitude",
            "Longitude",
            "Balancing Authority Code",
        ],
        index_col="Plant Code",
    )

    # filter
    eia_860_plant_df = eia_860_plant_df[
        eia_860_plant_df["Balancing Authority Code"] == bal_auth
    ]

    return eia_860_plant_df

# parse eia 860 plants (selecting by balancing authority)
eia_860_plant_file = Path('sample_data', '2___Plant_Y2019.xlsx')
eia_860_plants = load_eia_860_plants(eia_860_plant_file, 'CISO')

In [5]:
def load_eia_860_generators(
    eia_860_generator_file: Path,
    eia_860_plants: pd.DataFrame,
    additional_cols: list=[],
    tech_filter: list=[],
    invert_tech_filter: bool=False
    ) -> tuple[list, list, list]:
    """Return a list of thermal generator capacities, latitudes, and longitudes"""
    # read file
    eia_860_generator_df = pd.read_excel(
        eia_860_generator_file,
        skiprows=1,
        usecols=[
            "Plant Code",
            "Technology",
            "Nameplate Capacity (MW)",
            "Status"
        ] + additional_cols,
    )

    # filter by plants
    eia_860_generator_df = eia_860_generator_df[
        eia_860_generator_df["Plant Code"].isin(eia_860_plants.index)
    ]

    # filter by technology
    if tech_filter:
        if invert_tech_filter:
            eia_860_generator_df = eia_860_generator_df[
                ~eia_860_generator_df["Technology"].isin(
                    tech_filter
                )
            ]
        else:
            eia_860_generator_df = eia_860_generator_df[
                eia_860_generator_df["Technology"].isin(
                    tech_filter
                )
            ]

    # filter by status
    eia_860_generator_df = eia_860_generator_df[
        eia_860_generator_df["Status"] == "OP"
    ]

    eia_860_generator_df["Latitude"] = eia_860_generator_df["Plant Code"].map(lambda plant_code: eia_860_plants["Latitude"][plant_code])
    eia_860_generator_df["Longitude"] = eia_860_generator_df["Plant Code"].map(lambda plant_code: eia_860_plants["Longitude"][plant_code])

    return eia_860_generator_df

# parse eia 860 generator types
EIA_860_NON_THERMAL_TECHNOLOGY = [
    "Onshore Wind Turbine",
    "Conventional Hydroelectric",
    "Solar Photovoltaic",
    "Offshore Wind Turbine",
    "Batteries",
    "Hydroelectric Pumped Storage"
]
eia_860_generator_file = Path('sample_data', '3_1_Generator_Y2019.xlsx')
eia_860_wind_file = Path('sample_data', '3_2_Wind_Y2019.xlsx')
eia_860_solar_file = Path('sample_data', '3_3_Solar_Y2019.xlsx')
eia_860_storage_file = Path('sample_data', '3_4_Energy_Storage_Y2019.xlsx')

eia_860_thermal_generators = load_eia_860_generators(
    eia_860_generator_file, 
    eia_860_plants,
    tech_filter=EIA_860_NON_THERMAL_TECHNOLOGY,
    invert_tech_filter=True
)
eia_860_wind_generators = load_eia_860_generators(
    eia_860_wind_file,
    eia_860_plants
)
eia_860_solar_generators = load_eia_860_generators(
    eia_860_solar_file,
    eia_860_plants
)
eia_860_storage_generators = load_eia_860_generators(
    eia_860_storage_file,
    eia_860_plants,
    additional_cols=["Nameplate Energy Capacity (MWh)"]
)

We also need hourly solar and wind capacity factors and temperature profiles. We use another ASSET lab tool to generate sample data for CISO.

**Source:** https://github.com/ijbd/merra-power-generation


In [6]:
INT_TO_PCT = 1 / 100

# load processed power generation dataset (solar cf, wind cf, and temperature)
pow_gen_file = Path('sample_data','merra_power_generation_ciso_2019.nc')
pow_gen_dataset = xr.open_dataset(pow_gen_file)

# load temperature dependent outage rate (tdfor) table
tdfor_table_file = Path('sample_data', 'temperature_dependent_outage_rates.csv')
tdfor_table = pd.read_csv(tdfor_table_file, index_col=0)
tdfor_table = tdfor_table * INT_TO_PCT

# create mapping table for tdfor table
tech_categories = {
    "CC" : ["Natural Gas Fired Combined Cycle"],
    "CT" : ["Natural Gas Fired Combustion Turbine","Landfill Gas"],
    "DS" : ["Natural Gas Internal Combustion Engine"],
    "ST" : ["Conventional Steam Coal","Natural Gas Steam Turbine"],
    "NU" : ["Nuclear"],
    "HD" : ["Conventional Hydroelectric","Solar Thermal without Energy Storage",
                   "Hydroelectric Pumped Storage","Solar Thermal with Energy Storage","Wood/Wood Waste Biomass"]
}

# create mapping from technology to category
tech_mapping = {tech : cat for cat, techs in tech_categories.items() for tech in techs}


For conventional thermal generators, we map hourly temperature profiles into temperature-dependent forced outage rates, then instantiate thermal units as `assetra.units.StochasticUnit` objects. 

In [7]:
from assetra.units import StochasticUnit

def get_hourly_temperature(latitude:float, longitude:float, temperature_array: xr.DataArray) -> xr.DataArray:
    return temperature_array.sel(
            lat=latitude, 
            lon=longitude, 
            method='nearest').squeeze(drop=True)

def get_hourly_forced_outage_rate(hourly_temperature: xr.DataArray, technology: str) -> xr.DataArray:
    # index tdfor table by tech
    tdfor_map = tdfor_table[tech_mapping.get(technology, 'Other')]
    return xr.apply_ufunc(
        lambda hourly_temperature: tdfor_map.iloc[
            tdfor_map.index.get_indexer(hourly_temperature, method="nearest")
        ],
        hourly_temperature
    ).rename("hourly_forced_outage_rate")

for _, generator in eia_860_thermal_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_hourly_temperature(generator["Latitude"], generator["Longitude"], pow_gen_dataset['temperature'])

    # map temperature to hourly forced outage rate

    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # create assetra energy unit
    thermal_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator['Nameplate Capacity (MW)'],
            hourly_capacity=xr.ones_like(hourly_temperature).rename('hourly_capacity') * generator['Nameplate Capacity (MW)'],
            hourly_forced_outage_rate=hourly_forced_outage_rate
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(thermal_unit)

For solar and wind generators, we additionally scale nameplate capacities by hourly capacity factors from our MERRA power generation dataset. Solar and wind generators are added as `assetra.units.StochasticUnit` objects.


In [8]:
def get_hourly_capacity(
    nameplate_capacity: float,
    latitude: float,
    longitude: float,
    cf_array: xr.DataArray
    ) -> xr.DataArray:
    return nameplate_capacity*cf_array.sel(
            lat=latitude, 
            lon=longitude, 
            method='nearest'
        ).squeeze(drop=True)

# add solar
for _, generator in eia_860_solar_generators.iterrows():

    hourly_temperature = get_hourly_temperature(
        generator["Latitude"],
        generator["Longitude"],
        pow_gen_dataset["temperature"]
    )
    hourly_capacity = get_hourly_capacity(
        generator["Nameplate Capacity (MW)"],
        generator["Latitude"],
        generator["Longitude"],
        pow_gen_dataset["solar_capacity_factor"]
    )

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # create assetra energy unit
    solar_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator['Nameplate Capacity (MW)'],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(solar_unit)

# add wind
for _, generator in eia_860_wind_generators.iterrows():

    hourly_temperature = get_hourly_temperature(
        generator["Latitude"],
        generator["Longitude"],
        pow_gen_dataset["temperature"]
    )
    hourly_capacity = get_hourly_capacity(
        generator["Nameplate Capacity (MW)"],
        generator["Latitude"],
        generator["Longitude"],
        pow_gen_dataset["wind_capacity_factor"]
    )

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # create assetra energy unit
    wind_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator['Nameplate Capacity (MW)'],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(wind_unit)

Lastly, storage units are added as `assetra.units.StorageUnit` objects.

### 3. Build Energy System

Once all units have been added, the system builder can instantiate an `assetra.system.EnergySystem` object.

In [9]:
energy_system = builder.build()

### 4. Saving and Loading an Energy System

Energy systems can be saved to a local directory for re-use.

In [11]:
save_system_dir = Path("sample_data", "sample_energy_sys")
energy_system.save(save_system_dir)

If we have already built an energy system, this feature offers a nice starting point for iterative simulations.

In [1]:
from assetra.system import EnergySystem
from pathlib import Path

save_system_dir = Path("sample_data", "sample_energy_sys")
energy_system = EnergySystem()

energy_system.load(save_system_dir)

### 5. Running Probabilistic Simulations

In [2]:
from assetra.simulation import ProbabilisticSimulation
from logging import basicConfig, DEBUG

basicConfig(level=DEBUG)
simulation = ProbabilisticSimulation(
    start_hour="2019-01-01 00:00:00",
    end_hour="2019-12-31 23:00:00",
    trial_size=1000
)

simulation.assign_energy_system(energy_system)
simulation.run()

INFO:assetra.units:Using chunk size 12
DEBUG:assetra.units:Sampling outages for units 0-12 of 1746
DEBUG:assetra.units:Sampling outages for units 12-24 of 1746
DEBUG:assetra.units:Sampling outages for units 24-36 of 1746
DEBUG:assetra.units:Sampling outages for units 36-48 of 1746
DEBUG:assetra.units:Sampling outages for units 48-60 of 1746
DEBUG:assetra.units:Sampling outages for units 60-72 of 1746
DEBUG:assetra.units:Sampling outages for units 72-84 of 1746
DEBUG:assetra.units:Sampling outages for units 84-96 of 1746
DEBUG:assetra.units:Sampling outages for units 96-108 of 1746
DEBUG:assetra.units:Sampling outages for units 108-120 of 1746
DEBUG:assetra.units:Sampling outages for units 120-132 of 1746
DEBUG:assetra.units:Sampling outages for units 132-144 of 1746
DEBUG:assetra.units:Sampling outages for units 144-156 of 1746
DEBUG:assetra.units:Sampling outages for units 156-168 of 1746
DEBUG:assetra.units:Sampling outages for units 168-180 of 1746
DEBUG:assetra.units:Sampling outag